In [1]:
import $exclude.`org.slf4j:slf4j-log4j12`, $ivy.`org.slf4j:slf4j-nop:1.7.21` // for cleaner logs
import $profile.`hadoop-2.6`
import $ivy.`org.apache.spark::spark-sql:2.1.0` // adjust spark version - spark >= 2.0
import $ivy.`org.apache.spark::spark-mllib:2.1.0`
import $ivy.`org.apache.hadoop:hadoop-aws:2.6.4`
import $ivy.`org.jupyter-scala::spark:0.4.2` // for JupyterSparkSession (SparkSession aware of the jupyter-scala kernel)

import org.apache.spark._
import org.apache.spark.sql._
import jupyter.spark.session._

import org.apache.spark.graphx._
import org.apache.spark.rdd.RDD

import org.apache.spark.graphx.{Graph, VertexRDD}
import org.apache.spark.graphx.util.GraphGenerators

val sparkSession = JupyterSparkSession.builder() // important - call this rather than SparkSession.builder()
                    .jupyter()
                    .master("local[*]")
                    .appName("notebook")
                    .getOrCreate()
val sc = sparkSession.sparkContext

log4j:WARN No appenders could be found for logger (io.netty.util.internal.logging.InternalLoggerFactory).
log4j:WARN Please initialize the log4j system properly.
log4j:WARN See http://logging.apache.org/log4j/1.2/faq.html#noconfig for more info.


import $exclude.$                        , $ivy.$                            // for cleaner logs

import $profile.$           

import $ivy.$                                   // adjust spark version - spark >= 2.0

import $ivy.$                                    

import $ivy.$                                   

import $ivy.$                                // for JupyterSparkSession (SparkSession aware of the jupyter-scala kernel)


import org.apache.spark._

import org.apache.spark.sql._

import jupyter.spark.session._


import org.apache.spark.graphx._

import org.apache.spark.rdd.RDD


import org.apache.spark.graphx.{Graph, VertexRDD}

import org.apache.spark.graphx.util.GraphGenerators


sparkSession: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@69e9d79b
sc: org.apache.spark.SparkContext = org.apache.spark.SparkContext@153676b9

In [2]:
// Create an RDD for the vertices
val users: RDD[(VertexId, (String, String))] =
  sc.parallelize(Array((3L, ("rxin", "student")), (7L, ("jgonzal", "postdoc")),
                       (5L, ("franklin", "prof")), (2L, ("istoica", "prof"))))
// Create an RDD for edges
val relationships: RDD[Edge[String]] =
  sc.parallelize(Array(Edge(3L, 7L, "collab"),    Edge(5L, 3L, "advisor"),
                       Edge(2L, 5L, "colleague"), Edge(5L, 7L, "pi")))
// Define a default user in case there are relationship with missing user
val defaultUser = ("John Doe", "Missing")
// Build the initial Graph
val graph = Graph(users, relationships, defaultUser)

users: RDD[(VertexId, (String, String))] = ParallelCollectionRDD[0] at parallelize at cmd1.sc:2
relationships: RDD[Edge[String]] = ParallelCollectionRDD[1] at parallelize at cmd1.sc:6
defaultUser: (String, String) = ("John Doe", "Missing")
graph: Graph[(String, String), String] = org.apache.spark.graphx.impl.GraphImpl@785656cc

In [3]:
// Count all users which are postdocs
graph.vertices.filter { case (id, (name, pos)) => pos == "postdoc" }.count
// Count all the edges where src > dst
graph.edges.filter(e => e.srcId > e.dstId).count
graph.edges.filter { case Edge(src, dst, prop) => src > dst }.count

res2_0: Long = 1L
res2_1: Long = 1L
res2_2: Long = 1L

In [4]:
// Use the triplets view to create an RDD of facts.
val facts: RDD[String] = graph.triplets.map(triplet =>
    triplet.srcAttr._1 + " is the " + triplet.attr + " of " + triplet.dstAttr._1)
facts.collect.foreach(println(_))

rxin is the collab of jgonzal
franklin is the advisor of rxin
istoica is the colleague of franklin
franklin is the pi of jgonzal


facts: RDD[String] = MapPartitionsRDD[24] at map at cmd3.sc:1

In [5]:
// Remove missing vertices as well as the edges to connected to them
val validGraph = graph.subgraph(vpred = (id, attr) => attr._2 != "Missing")
// The valid subgraph will disconnect users 4 and 5 by removing user 0
validGraph.vertices.collect.foreach(println(_))
validGraph.triplets.map(
  triplet => triplet.srcAttr._1 + " is the " + triplet.attr + " of " + triplet.dstAttr._1
).collect.foreach(println(_))

(2,(istoica,prof))
(3,(rxin,student))
(5,(franklin,prof))
(7,(jgonzal,postdoc))
rxin is the collab of jgonzal
franklin is the advisor of rxin
istoica is the colleague of franklin
franklin is the pi of jgonzal


validGraph: Graph[(String, String), String] = org.apache.spark.graphx.impl.GraphImpl@2e330047

In [6]:
// Run Connected Components
val ccGraph = graph.connectedComponents() // No longer contains missing field
// Remove missing vertices as well as the edges to connected to them
val validGraph = graph.subgraph(vpred = (id, attr) => attr._2 != "Missing")
// Restrict the answer to the valid subgraph
val validCCGraph = ccGraph.mask(validGraph)

ccGraph: Graph[VertexId, String] = org.apache.spark.graphx.impl.GraphImpl@15184cc4
validGraph: Graph[(String, String), String] = org.apache.spark.graphx.impl.GraphImpl@40fc6bc2
validCCGraph: Graph[VertexId, String] = org.apache.spark.graphx.impl.GraphImpl@547e4cb8

In [7]:
val outDegrees: VertexRDD[Int] = graph.outDegrees
val degreeGraph = graph.outerJoinVertices(outDegrees) { (id, oldAttr, outDegOpt) =>
  outDegOpt match {
    case Some(outDeg) => outDeg
    case None => 0 // No outDegree means zero outDegree
  }
}

outDegrees: VertexRDD[Int] = VertexRDDImpl[97] at RDD at VertexRDD.scala:57
degreeGraph: Graph[Int, String] = org.apache.spark.graphx.impl.GraphImpl@49d12b5c

In [8]:
// Create a graph with "age" as the vertex property.
// Here we use a random graph for simplicity.
val graph: Graph[Double, Int] =
  GraphGenerators.logNormalGraph(sc, numVertices = 100).mapVertices( (id, _) => id.toDouble )
// Compute the number of older followers and their total age
val olderFollowers: VertexRDD[(Int, Double)] = graph.aggregateMessages[(Int, Double)](
  triplet => { // Map Function
    if (triplet.srcAttr > triplet.dstAttr) {
      // Send message to destination vertex containing counter and age
      triplet.sendToDst(1, triplet.srcAttr)
    }
  },
  // Add counter and age
  (a, b) => (a._1 + b._1, a._2 + b._2) // Reduce Function
)
// Divide total age by number of older followers to get average age of older followers
val avgAgeOfOlderFollowers: VertexRDD[Double] =
  olderFollowers.mapValues( (id, value) =>
    value match { case (count, totalAge) => totalAge / count } )
// Display the results
avgAgeOfOlderFollowers.collect.foreach(println(_))

(96,97.0)
(56,77.28571428571429)
(16,62.04761904761905)
(80,88.42857142857143)
(48,73.33333333333333)
(32,72.42857142857143)
(0,48.333333333333336)
(24,58.94736842105263)
(64,82.2)
(40,76.70588235294117)
(72,86.6)
(8,59.7)
(88,94.5)
(41,73.22222222222223)
(81,88.4)
(25,64.94117647058823)
(65,75.5)
(73,89.0)
(57,76.5)
(33,64.0909090909091)
(1,45.029411764705884)
(89,93.66666666666667)
(17,61.5)
(9,60.130434782608695)
(49,80.46666666666667)
(34,71.75)
(82,94.33333333333333)
(66,78.75)
(50,72.23076923076923)
(42,72.93333333333334)
(74,85.71428571428571)
(90,95.0)
(18,52.05263157894737)
(58,77.6)
(26,57.294117647058826)
(10,47.8)
(2,48.61290322580645)
(19,66.78571428571429)
(59,72.5)
(11,55.76923076923077)
(35,67.63636363636364)
(27,68.875)
(75,88.66666666666667)
(51,73.8125)
(83,91.0)
(67,77.55555555555556)
(3,42.78260869565217)
(43,72.5)
(84,92.5)
(52,71.3076923076923)
(4,49.32142857142857)
(76,92.0)
(28,67.33333333333333)
(36,73.84210526315789)
(92,95.66666666666667)
(20,56.55)
(12,51.2

graph: Graph[Double, Int] = org.apache.spark.graphx.impl.GraphImpl@44b0354d
olderFollowers: VertexRDD[(Int, Double)] = VertexRDDImpl[128] at RDD at VertexRDD.scala:57
avgAgeOfOlderFollowers: VertexRDD[Double] = VertexRDDImpl[130] at RDD at VertexRDD.scala:57

In [9]:
// Define a reduce operation to compute the highest degree vertex
def max(a: (VertexId, Int), b: (VertexId, Int)): (VertexId, Int) = {
  if (a._2 > b._2) a else b
}
// Compute the max degrees
val maxInDegree: (VertexId, Int)  = graph.inDegrees.reduce(max)
val maxOutDegree: (VertexId, Int) = graph.outDegrees.reduce(max)
val maxDegrees: (VertexId, Int)   = graph.degrees.reduce(max)

defined function max
maxInDegree: (VertexId, Int) = (10L, 39)
maxOutDegree: (VertexId, Int) = (30L, 96)
maxDegrees: (VertexId, Int) = (30L, 125)